## Notebook 1
# Extraindo coordenadas de decretos e leis

Autores:
* Pedro C. de Siracusa
* André G. Coutinho
* Waira Saravia Machida
* Augusto Arcela
* Yan Felipe Soares

---

Neste estudo exploramos decretos e leis que embasam a criação de UCs, em âmbitos Federal e Municipal.
O objetivo é extrair as coordenadas que delimitam as UCs, de forma que possamos posteriormente projetar o polígono em um ambiente SIG. 
Os **principais desafios**:

* Os documentos estão no formato `.pdf`, não muito amigável para a extração de dados. Algumas bibliotecas auxiliam na extração de texto de arquivos `.pdf`, mas elas nem sempre funcionam como esperamos. Isso porque estes arquivos podem ter sido codificados de muitas formas diferentes, e nem sempre conseguimos fazer a extração.

* Os documentos não seguem um padrão estrutural, o que significa que nem sempre a informação que desejamos está no mesmo lugar ou se apresenta da mesma forma. Isso dificulta a extração inteiramente automatizada da informação nestes documentos.

* A delimitação das UCs é apresentada de forma descritiva nestes documentos. Em alguns poucos casos as coordenadas são organizadas em uma tabela, facilitando um pouco a extração.

Vamos começar explorando o **Decreto No.37826** da Assembleia de Minas Gerais, que altera a categoria de manejo da Reserva Biológica de Corumbá e fixa seus limites.
O documento está nomeado com o`d37826.pdf`, localizado dentro do diretório `data`, na raiz do projeto.

O primeiro passo foi extrair (manualmente) a parte do documento que descreve a delimitação da UC.
Armazenamos esta informação no arquivo `d37826_limits.txt`, para facilitar a leitura e extração de coordenadas do texto.

In [8]:
fpath = '../data/decretos/limits/d37826_limits.txt'

Agora vamos abrir o arquivo, ler as linhas de texto e armazenar numa variável `lines`. Na célula abaixo usamos uma estrutura do *Python* que se chama [context manager](https://docs.python.org/3/reference/compound_stmts.html#with), para facilitar a abertura/fechamento de arquivos.

In [9]:
with open(fpath, encoding='utf-8') as f:
    lines =''.join( f.readlines() )

Vejamos os 1000 primeiros caracteres do texto:

In [18]:
lines[:1000]+' (...)'

'Memorial descritivo de medição e demarcação da Estação Ecológica Corumbá, localizada à margem da Rodovia Corumbá/Arcos, nas proximidades do trevo para a COMIG, no lugar denominado Corumbá, cuja gleba e delimitada por um polígono irregular. O ponto de partida foi localizado no marco cravado na distância de 1,00 m, à esquerda do canto da cerca de divisa entre a Estação Ecológica, a Rodovia Arcos—Corumbá e Rolando Alves de Souza, com as coordenadasabsolutas E-10.000,000 e N-5.000,000; daí, segue confrontando com Rolando Alves de Souza, com o azimute de 298238\'00", na distância de 296,103 m, até atingir o vértice nº 01, de coordenadas E-9.740,175 e N-5.141,866; segue pela cerca de divisa à esquerda, na distância de 1,50 m; segue a ré, na distância de 276,00 m, pelo Córrego Santo Antônio, e sai; segue a ré, na distância de 193,00 m, e cruza a rede telefônica; daí, segue com o azimute de 301°42\'124", na distância de 128,861 m, até atingir o vértice nº 02, de coordenadas E-9.633,126 e N-5.

Neste caso, as coordenadas parecem estar descritas em UTM seguindo o padrão `E-X.XXX,XXX e N-X.XXX,XXX`.
Para extrair o texto que segue este padrão vamos usar **expressões regulares**.

---

## Regular Expressions

Expressões regulares servem para definirmos padrões de busca em um texto. Uma vez definido o padrão que desejamos buscar, o texto é percorrido tentando encontrar "casamentos", que podem então ser retornados. Usaremos expressões regulares para tentar extrair as coordenadas, assumindo que elas tenham sido inseridas em um padrão relativamente estável.

Alguns exemplos e guias de como se utilizar expressões regulares: 
1. https://medium.com/tech-tajawal/regular-expressions-the-last-guide-6800283ac034
1. https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285
1. http://www.rexegg.com/regex-quickstart.html#ref

O *Python* já fornece um módulo que nos permite trabalhar com expressões regulares de forma facilitada. O nome do módulo é `re`, e para usá-lo precisamos importá-lo. Veja [aqui](https://docs.python.org/3/howto/regex.html) como usar expressões regulares com o módulo `re`.

In [22]:
import re

#### Primeira tentativa

Abaixo vamos fazer uma primeira tentativa de expressão para capturar as coordenadas. Guardamos a expressão em uma variável de nome `expr`.

In [19]:
expr = '[EN]\-?[0-9\.\-,]+,[0-9]{1,3}'

Agora vamos usar a função `findall` para encontrar todos os *"matches"* para esta expressão. As coordenadas obtidas são armazenadas na variável `coords`. Para melhorar a visualização vou olhar apenas para as 10 primeiras coordenadas que foram obtidas.

In [23]:
coords = re.findall( expr , lines )

In [25]:
coords[:10]

['E-10.000,000',
 'N-5.000,000',
 'E-9.740,175',
 'N-5.141,866',
 'E-9.633,126',
 'N-5.208,003',
 'E-9.589,200',
 'N-5.238,457',
 'E-9.661,851',
 'N-5.426,092']

Quantas coordenadas foram obtidas ao todo?

In [26]:
len(coords)

228

Um problema deste método que usamos acima é que fica um pouco difícil validar se as coordenadas realmente estão sendo extraídas em pares. Caso uma delas não tenha sido capturada, isso pode gerar uma deformação do polígono extraído.

#### Segunda tentativa

Vamos fazer uma segunda tentativa. Agora, queremos incluir também a palavra 'coordenadas', que parece sempre indicar um novo par de coordenadas no texto original.

In [34]:
expr = 'coordenadas [EN]\-?[0-9\.\-,]+,[0-9]{1,3} e [EN]\-?[0-9\.\-,]+,[0-9]{1,3}'

In [35]:
coords = re.findall( expr, lines )

In [36]:
coords[:10] # observando apenas as primeiras 10 entradas

['coordenadas E-9.740,175 e N-5.141,866',
 'coordenadas E-9.633,126 e N-5.208,003',
 'coordenadas E-9.589,200 e N-5.238,457',
 'coordenadas E-9.661,851 e N-5.426,092',
 'coordenadas E-9.634,180 e N-5.422,209',
 'coordenadas E-9.712,095 e N5.551,784',
 'coordenadas E-9.696,339 e N-5.553,894',
 'coordenadas E-9.719,684 e N-5.706,781',
 'coordenadas E-9.170,600 e N-5.598,175',
 'coordenadas E9.668,726 e N-5.632,721']

No texto original, existem no total 83 vértices. Quantos vértices conseguimos extrair com o método acima?

In [37]:
len(coords)

113

Isso aconteceu porque, além da numeração de vértices, a pessoa que elaborou o documento também descreveu coordenadas usando outros termos que descrevem coordenadas, como 'irradiação'. Optamos por usar também estas coordenadas.

---

## Estruturando coordenadas

Agora que temos uma lista de strings armazenando coordenadas na forma `coordenadas E-X.XXX,XX e N-X.XXX,XXX`, precisamos extrair apenas os valores de coordenadas propriamente ditas.
Como cada coordenada é formada por um par de valores, vamos representá-las como tuplas. A função abaixo realiza esta tarefa:

In [39]:
def getCoords(coordsStr):
    coordsStrSplit = coordsStr.split(' ')
    return ( coordsStrSplit[1], coordsStrSplit[-1] )

Sendo assim, podemos obter uma lista de pares ordenados, cada qual formando uma coordenada. Aplicaremos a função `getCoords` sobre cada elemento da lista na variável `coords`. Ao final, sobreescreveremos a variável `coords` para conter a nova lista de tuplas (em vez da antiga lista de strings).

In [44]:
coords = [ getCoords(c) for c in coords ]

> nota: a estrutura acima é nativa do *Python*, e é conhecida como "compreensão de listas" (**list comprehension**). Ela permite a construção de listas de forma bastante rápida, usando expressões 'for'.

Vejamos os 10 primeiros valores que obtivemos:

In [46]:
coords[:10]

[('E-9.740,175', 'N-5.141,866'),
 ('E-9.633,126', 'N-5.208,003'),
 ('E-9.589,200', 'N-5.238,457'),
 ('E-9.661,851', 'N-5.426,092'),
 ('E-9.634,180', 'N-5.422,209'),
 ('E-9.712,095', 'N5.551,784'),
 ('E-9.696,339', 'N-5.553,894'),
 ('E-9.719,684', 'N-5.706,781'),
 ('E-9.170,600', 'N-5.598,175'),
 ('E9.668,726', 'N-5.632,721')]

Agora precisamos validar e corrigir as coordenadas na lista `coords`.

---

## Validando as coordenadas

In [58]:
fixDecimal = lambda x: x.replace('.','').replace(',','.')

In [59]:
[ (fixDecimal(e), fixDecimal(n)) for e,n in coords ]

[('E-9740.175', 'N-5141.866'),
 ('E-9633.126', 'N-5208.003'),
 ('E-9589.200', 'N-5238.457'),
 ('E-9661.851', 'N-5426.092'),
 ('E-9634.180', 'N-5422.209'),
 ('E-9712.095', 'N5551.784'),
 ('E-9696.339', 'N-5553.894'),
 ('E-9719.684', 'N-5706.781'),
 ('E-9170.600', 'N-5598.175'),
 ('E9668.726', 'N-5632.721'),
 ('E-9706.017', 'N-5696.076'),
 ('E-9689.773', 'N-5735.972'),
 ('E-906.089', 'N-5804.720'),
 ('E-9575.597', 'N-6053.718'),
 ('E-9572.885', 'N-6076.549'),
 ('E-9534.847', 'N6068.565'),
 ('E-9507.959', 'N-6230.886'),
 ('E-9499.481', 'N-6085.920'),
 ('E-9444.098', 'N-5979.095'),
 ('E-9363.137', 'N-5901.473'),
 ('E-367.445', 'N5956.872'),
 ('E-9382.204', 'N-6072.590'),
 ('E-9408.169', 'N-6197.588'),
 ('E-9352.294', 'N-6343.442'),
 ('E-9267.145', 'N-6545.753'),
 ('E-9300.187', 'N-4462.741'),
 ('E-9235.441', 'N-6251.967'),
 ('E-9200.010', 'N-6602.572'),
 ('E-9124.414', 'N-6679.944'),
 ('E-8843.971', 'N-6913.206'),
 ('E-8934.036', 'N-6815.326'),
 ('E-8827.770', 'N6784.257'),
 ('E-8759.079',